In [4]:
from skf.util._database import connect_sql,read_data,update_data, disconnect_sql
import json

config = json.load(open('config.json'))
connection = connect_sql(config.get('db_host'), config.get('db_user'), config.get('db_password'), config.get('db_name'))
sList = ['WV3333', '13/Feb/2024 03:42:46', 'GET', '/Windchill/netmarkets/jsp/util/getBundleKey.jsp?key=com.ptc.netmarkets.util.utilResource.CONFIRM_EXTERNAL_SITE', '200']
query = ("INSERT INTO apachelog VALUES %s;" % (tuple(sList),))
print(query)

# query = "DELETE FROM apachelog WHERE userid='WV3333'"

update_data(connection,query)

query = 'SELECT * from apachelog'

rows= read_data(connection,query)

disconnect_sql(connection)

print(rows)

INSERT INTO apachelog VALUES ('WV3333', '13/Feb/2024 03:42:46', 'GET', '/Windchill/netmarkets/jsp/util/getBundleKey.jsp?key=com.ptc.netmarkets.util.utilResource.CONFIRM_EXTERNAL_SITE', '200');
[{'userid': 'WV3333', 'accessdate': datetime.datetime(2024, 2, 13, 3, 42, 46), 'httpmethod': 'GET', 'query': '/Windchill/netmarkets/jsp/util/getBundleKey.jsp?key=com.ptc.netmarkets.util.utilResource.CONFIRM_EXTERNAL_SITE', 'httpstatus': '200'}, {'userid': 'WV3333', 'accessdate': datetime.datetime(2024, 2, 13, 3, 42, 46), 'httpmethod': 'GET', 'query': '/Windchill/netmarkets/jsp/util/getBundleKey.jsp?key=com.ptc.netmarkets.util.utilResource.CONFIRM_EXTERNAL_SITE', 'httpstatus': '200'}]


In [4]:
import os
import json
from skf.util import process_apache_logs

config = json.load(open('config.json'))
log_path = 'C:\\temp\\WcProd_ApacheLogs\\accesslogs\\W2750' #config.get('logs_path')

# logdata = process_apache_logs(log_path)

logdata = []
files = os.listdir(log_path)

for f in files:
    fpath = log_path+"/"+f
    with open(fpath, 'r') as f:
        for row in f.readlines():
            if 'wt.httpgw.HTTPServer' not in row:
                cleanedLogvalue = row.strip().replace("[","").replace("]","").replace("\"","").split()
                cleanedLogvalue[3] = cleanedLogvalue[3].replace(':',' ',1)
                cleanedLogvalue.pop(4)
                cleanedLogvalue.pop(6)
                templist = cleanedLogvalue[2:7]
                if '/servlet' in cleanedLogvalue[5]:
                    technology = cleanedLogvalue[5].split('servlet/')[1].split("/")[0].split('?')[0]
                    if len(technology) > 50:
                        technology = technology[:50]
                    templist.append(technology)
                    if 'servlet/IE' in cleanedLogvalue[5] and '/tasks' in cleanedLogvalue[5]:
                        wccustomization = cleanedLogvalue[5].split('tasks')[1].split("?")[0]
                        endpoint = cleanedLogvalue[5].split('tasks')[1].split("?")[0].split('/')[-1]
                        wccustomization = wccustomization.replace(f'/{endpoint}',"")
                        templist.append(wccustomization)
                        templist.append(endpoint)
                    elif 'servlet/odata' in cleanedLogvalue[5]:
                        wccustomization = (cleanedLogvalue[5].split('odata')[1].split("?")[0])
                        endpoint = (cleanedLogvalue[5].split('odata')[1].split("?")[0].split('/')[-1])
                        wccustomization = wccustomization.replace(f'/{endpoint}',"")
                        templist.append(wccustomization.split("(")[0])
                        templist.append(endpoint.split("(")[0]) # Added split to avoid object specific info in endpoint
                    elif 'servlet/rest' in cleanedLogvalue[5]:
                        wccustomization = (cleanedLogvalue[5].split('rest')[1].split("?")[0])
                        endpoint = (cleanedLogvalue[5].split('rest')[1].split("?")[0].split('/')[-1])
                        wccustomization = wccustomization.replace(f'/{endpoint}',"")
                        templist.append(wccustomization)
                        templist.append(endpoint)
                    else:   # To manage database update empty values added
                        templist.append('')
                        templist.append('')
                elif 'infoengine' in cleanedLogvalue[5] and '/jsp' in cleanedLogvalue[5]:
                    technology = 'infoengine'
                    templist.append(technology)
                    wccustomization = (cleanedLogvalue[5].split('/jsp')[1].split("?")[0])
                    endpoint = (cleanedLogvalue[5].split('/jsp')[1].split("?")[0].split('/')[-1])
                    wccustomization = wccustomization.replace(f'/{endpoint}',"")
                    templist.append(wccustomization)
                    templist.append(endpoint)
                else:
                    templist.append('') # To manage datbase update empty values added
                    templist.append('')
                    templist.append('')
                # print(templist)
                logdata.append(templist)
                #print(templist)

    # break

# files_with_path = [os.path.abspath(x) for x in os.listdir(log_path)]
print(len(logdata))
print(logdata[1:10])


521054
[['-', '31/Jan/2024 00:00:00', 'GET', '/', '200', '', '', ''], ['-', '31/Jan/2024 00:00:01', 'GET', '/', '200', '', '', ''], ['G002269', '31/Jan/2024 00:00:03', 'POST', '/Windchill/servlet/WindchillAuthGW/com.ptc.windchill.uwgm.soap.impl.uwgm.BinaryRequestHandler/getCacheDescriptor?trxid=29059922722834.16&SessionId=G002269@100.64.0.1_2024-01-30T19:56:23.812-03:0&action_id=WorkspaceTableView_20240130200001_29059922722834&ReqNum=16&cancelModelConv=true', '200', 'WindchillAuthGW', '', ''], ['-', '31/Jan/2024 00:00:04', 'GET', '/', '200', '', '', ''], ['-', '31/Jan/2024 00:00:04', 'GET', '/Windchill/wtcore/images/akamai/sr-test-0043kb.html', '200', '', '', ''], ['-', '31/Jan/2024 00:00:04', 'GET', '/', '200', '', '', ''], ['-', '31/Jan/2024 00:00:05', 'GET', '/Windchill/wtcore/images/akamai/sr-test-0043kb.html', '200', '', '', ''], ['-', '31/Jan/2024 00:00:06', 'GET', '/', '200', '', '', ''], ['-', '31/Jan/2024 00:00:09', 'GET', '/', '200', '', '', '']]


In [20]:
technology=['REST','IE','Others']
customizations=['mupu','odata']
text = 'skf/wepi/v2_6p/WepiQuery1.xml'
sText = text.split("/")[::-1]

rtext = sText

print('/'.join(rtext[1:]))

# print((reversed(text)).split('/',1))

v2_6p/wepi/skf


In [6]:
import os

fPath = 'C:\\temp\\WcProd_ApacheLogs\\accesslogs'

files_path = [os.path.abspath(x) for x in os.listdir(fPath)]
print(os.listdir(files_path[0]))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\NZ8554\\OneDrive - SKF\\Documents\\Rajesh_D\\Development\\Python\\windchill-analytics\\W2749'